In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


import pandas as pd
import matplotlib.pyplot as plt

In [20]:
df = pd.read_csv('/home/fahd/Documents/headline-trader/headline-trader/src/training/full_df.csv')
df['label'].value_counts()


label
sell    3042
buy     2995
hold     686
Name: count, dtype: int64

In [14]:
df["label"].value_counts()
df.duplicated().sum()

features, labels

(      sentiment_score       price  hour_of_day  day_of_week
 0            0.854923  95478.3600           14            4
 1            0.902940  95478.3600           14            4
 2            0.740927  95478.3600           14            4
 3            0.939790  95478.3600           14            4
 4            0.910740  95478.3600           14            4
 ...               ...         ...          ...          ...
 6718         0.874792      2.1942            2            5
 6719         0.943423      2.1942            2            5
 6720         0.908315      2.1942            2            5
 6721         0.860967      2.1942            2            5
 6722         0.938050      2.1942            2            5
 
 [6723 rows x 4 columns],
 0       2
 1       2
 2       2
 3       2
 4       2
        ..
 6718    1
 6719    1
 6720    1
 6721    1
 6722    1
 Name: label, Length: 6723, dtype: int64)

In [23]:
# convert to pytorch tensors

x = torch.tensor(features.values, dtype=torch.float32)
y = torch.tensor(labels.values, dtype=torch.long)

x.shape[0]

6723

In [27]:
# Training and testing

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)



In [30]:
# Using DataLoader for batching 
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [31]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 3)  # 3 classes: buy/sell/hold

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)

model = MyModel(input_size=x.shape[1])

In [32]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:
epochs = 20
for epoch in range(epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        total_loss = 0
        outputs = model(batch_x) # calling the forward function
        loss = criterion(outputs, batch_y)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Avg Loss: {total_loss / len(train_loader):.4f}")

Epoch 1/20, Avg Loss: 0.0202
Epoch 2/20, Avg Loss: 0.1800
Epoch 3/20, Avg Loss: 0.0045
Epoch 4/20, Avg Loss: 0.0005
Epoch 5/20, Avg Loss: 0.2968
Epoch 6/20, Avg Loss: 0.0034
Epoch 7/20, Avg Loss: 0.0013
Epoch 8/20, Avg Loss: 0.0555
Epoch 9/20, Avg Loss: 0.1822
Epoch 10/20, Avg Loss: 0.0080
Epoch 11/20, Avg Loss: 0.0030
Epoch 12/20, Avg Loss: 0.0971
Epoch 13/20, Avg Loss: 0.0068
Epoch 14/20, Avg Loss: 0.0794
Epoch 15/20, Avg Loss: 0.1952
Epoch 16/20, Avg Loss: 0.6078
Epoch 17/20, Avg Loss: 0.0000
Epoch 18/20, Avg Loss: 0.1700
Epoch 19/20, Avg Loss: 0.0054
Epoch 20/20, Avg Loss: 0.0191


In [34]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")

Accuracy: 49.96%
